<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

参考：[Kaggle事始め](https://qiita.com/taka4sato/items/802c494fdebeaa7f43b7)

In [97]:
import pandas as pd
import csv as csv
from sklearn.ensemble import RandomForestClassifier

In [98]:
# Load training data
train_df = pd.read_csv("train.csv", header=0)

# Convert "Sex" to be a dummy variable (female = 0, Male = 1)
train_df["Gender"] = train_df["Sex"].map({"female": 0, "male": 1}).astype(int)
train_df.head(3)

# Convert "Embarked" to be a dummy variable (S = 0,C = 1, Q = 2)
train_df["Embarked"] = train_df["Embarked"].dropna().map({"S": 0, "C": 1, "Q": 2}).astype(int)
train_df.head(3)

# Complement the missing values of "Age" column with average of "Age"
median_age = train_df["Age"].dropna().median()
if len(train_df.Age[train_df.Age.isnull()]) > 0:
    train_df.loc[(train_df.Age.isnull()), "Age"] = median_age
    
# Complement the missing values of "Embarked" column with average of "Embarked"
median_emb = train_df["Embarked"].dropna().median()
if len(train_df.Embarked[train_df.Embarked.isnull()]) > 0:
    train_df.loc[(train_df.Embarked.isnull()), "Embarked"] = median_emb
    
# remove un-used columns
train_df = train_df.drop(["Name", "Ticket", "Sex", "Cabin", "PassengerId"], axis=1)
train_df.head(3)

,Survived,Pclass,Age,SibSp,Parch,Fare,Embarked,Gender
0,0,3,22.0,1,0,7.2500,0.0,1
1,1,1,38.0,1,0,71.2833,1.0,0
2,1,3,26.0,0,0,7.9250,0.0,0


In [99]:
# Load test data, Convert "Sex" to be a dummy variable
test_df = pd.read_csv("test.csv", header=0)
test_df["Gender"] = test_df["Sex"].map({"female": 0, "male": 1}).astype(int)

# Convert "Embarked" to be a dummy variable (S = 0,C = 1, Q = 2)
test_df["Embarked"] = test_df["Embarked"].dropna().map({"S": 0, "C": 1, "Q": 2}).astype(int)
test_df.head(3)

# Complement the missing values of "Age" column with average of "Age"
median_age = test_df["Age"].dropna().median()
if len(test_df.Age[test_df.Age.isnull()]) > 0:
    test_df.loc[(test_df.Age.isnull()), "Age"] = median_age
    
# Complement the missing values of "Embarked" column with average of "Embarked"
median_emb = test_df["Embarked"].dropna().median()
if len(test_df.Embarked[test_df.Embarked.isnull()]) > 0:
    test_df.loc[(test_df.Embarked.isnull()), "Embarked"] = median_emb
    
# Complement the missing values of "Fare" column with average of "Fare"
median_fare = test_df["Fare"].dropna().median()
if len(test_df.Fare[test_df.Fare.isnull()]) > 0:
    test_df.loc[(test_df.Fare.isnull()), "Fare"] = median_fare

# Copy test data's "PassengerId" column, and remove un-used columns
ids = test_df["PassengerId"].values
test_df = test_df.drop(["Name", "Ticket", "Sex", "Cabin", "PassengerId"], axis=1)
test_df.head(3)

,Pclass,Age,SibSp,Parch,Fare,Embarked,Gender
0,3,34.5,0,0,7.8292,2,1
1,3,47.0,1,0,7.0000,0,0
2,2,62.0,0,0,9.6875,2,1


In [100]:
train_x = train_df.values[:,1:]
train_y = train_df.values[:,0]

In [101]:
print("train_x:",train_x.shape,"train_y:",train_y.shape)

train_x: (891, 7) train_y: (891,)


In [102]:
from tensorflow.python.keras.utils import to_categorical

In [103]:
train_y = to_categorical(train_y)

In [104]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense,Dropout

In [111]:
def current_best_model():
    model.add(
        Dense(
            units=128, 
            input_shape=(7,),
            activation='relu'
            )
        )

    model.add(Dropout(0.25))

    model.add(
        Dense(
            units=64, 
            activation='relu'
        )
    )

    model.add(Dropout(0.25))

    model.add(
        Dense(
            units=32, 
            activation='relu'
        )
    )

    model.add(
        Dense(
            units=2, 
            activation='softmax'
        )
    )

In [156]:
def latest_model():
    model.add(
        Dense(
            units=14, 
            input_shape=(7,),
            activation='relu'
            )
    )

    model.add(
        Dense(
            units=14, 
            activation='relu'
        )
    )

    model.add(
        Dense(
            units=2, 
            activation='softmax'
        )
    )

In [157]:
model = Sequential()

In [158]:
#current_best_model()
latest_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_132 (Dense)            (None, 14)                112       
_________________________________________________________________
dense_133 (Dense)            (None, 14)                210       
_________________________________________________________________
dense_134 (Dense)            (None, 2)                 30        
Total params: 352
Trainable params: 352
Non-trainable params: 0
_________________________________________________________________


In [160]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history_adam = model.fit(
    train_x,
    train_y,
    batch_size=32,
    epochs=200,
    validation_split=0.2)

Train on 712 samples, validate on 179 samples
Epoch 1/200
712/712 [==============================] - 2s 3ms/step - loss: 0.4261 - acc: 0.8118 - val_loss: 0.4504 - val_acc: 0.8324
Epoch 2/200
712/712 [==============================] - 0s 108us/step - loss: 0.4397 - acc: 0.8034 - val_loss: 0.3657 - val_acc: 0.8492
Epoch 3/200
712/712 [==============================] - 0s 148us/step - loss: 0.4344 - acc: 0.8104 - val_loss: 0.3627 - val_acc: 0.8492
Epoch 4/200
712/712 [==============================] - 0s 134us/step - loss: 0.4196 - acc: 0.8076 - val_loss: 0.3395 - val_acc: 0.8547
Epoch 5/200
712/712 [==============================] - 0s 112us/step - loss: 0.4209 - acc: 0.8160 - val_loss: 0.3352 - val_acc: 0.8603
Epoch 6/200
712/712 [==============================] - 0s 115us/step - loss: 0.4205 - acc: 0.8160 - val_loss: 0.3396 - val_acc: 0.8659
Epoch 7/200
712/712 [==============================] - 0s 132us/step - loss: 0.4370 - acc: 0.8090 - val_loss: 0.3464 - val_acc: 0.8659
Epoch 8/200

Epoch 61/200
712/712 [==============================] - 0s 121us/step - loss: 0.4180 - acc: 0.8216 - val_loss: 0.3350 - val_acc: 0.8659
Epoch 62/200
712/712 [==============================] - 0s 158us/step - loss: 0.4194 - acc: 0.8132 - val_loss: 0.3528 - val_acc: 0.8547
Epoch 63/200
712/712 [==============================] - 0s 241us/step - loss: 0.4191 - acc: 0.8160 - val_loss: 0.3385 - val_acc: 0.8659
Epoch 64/200
712/712 [==============================] - 0s 272us/step - loss: 0.4249 - acc: 0.8076 - val_loss: 0.3347 - val_acc: 0.8603
Epoch 65/200
712/712 [==============================] - 0s 142us/step - loss: 0.4207 - acc: 0.8272 - val_loss: 0.3298 - val_acc: 0.8659
Epoch 66/200
712/712 [==============================] - 0s 108us/step - loss: 0.4129 - acc: 0.8090 - val_loss: 0.3522 - val_acc: 0.8436
Epoch 67/200
712/712 [==============================] - 0s 108us/step - loss: 0.4148 - acc: 0.8174 - val_loss: 0.3327 - val_acc: 0.8603
Epoch 68/200
712/712 [==========================

712/712 [==============================] - 0s 213us/step - loss: 0.4430 - acc: 0.8062 - val_loss: 0.3336 - val_acc: 0.8436
Epoch 122/200
712/712 [==============================] - 0s 236us/step - loss: 0.4324 - acc: 0.8034 - val_loss: 0.3631 - val_acc: 0.8547
Epoch 123/200
712/712 [==============================] - 0s 218us/step - loss: 0.4349 - acc: 0.8118 - val_loss: 0.3280 - val_acc: 0.8547
Epoch 124/200
712/712 [==============================] - 0s 189us/step - loss: 0.4127 - acc: 0.8258 - val_loss: 0.3362 - val_acc: 0.8603
Epoch 125/200
712/712 [==============================] - 0s 129us/step - loss: 0.4163 - acc: 0.8230 - val_loss: 0.3270 - val_acc: 0.8547
Epoch 126/200
712/712 [==============================] - 0s 113us/step - loss: 0.4218 - acc: 0.8188 - val_loss: 0.3349 - val_acc: 0.8659
Epoch 127/200
712/712 [==============================] - 0s 124us/step - loss: 0.4150 - acc: 0.8188 - val_loss: 0.3507 - val_acc: 0.8492
Epoch 128/200
712/712 [==============================] 

Epoch 181/200
712/712 [==============================] - 0s 114us/step - loss: 0.4118 - acc: 0.8216 - val_loss: 0.3481 - val_acc: 0.8380
Epoch 182/200
712/712 [==============================] - 0s 124us/step - loss: 0.4081 - acc: 0.8202 - val_loss: 0.3279 - val_acc: 0.8659
Epoch 183/200
712/712 [==============================] - 0s 121us/step - loss: 0.4155 - acc: 0.8090 - val_loss: 0.3564 - val_acc: 0.8492
Epoch 184/200
712/712 [==============================] - 0s 115us/step - loss: 0.4103 - acc: 0.8230 - val_loss: 0.3342 - val_acc: 0.8492
Epoch 185/200
712/712 [==============================] - 0s 114us/step - loss: 0.4340 - acc: 0.8020 - val_loss: 0.3360 - val_acc: 0.8547
Epoch 186/200
712/712 [==============================] - 0s 231us/step - loss: 0.4567 - acc: 0.8048 - val_loss: 0.3357 - val_acc: 0.8492
Epoch 187/200
712/712 [==============================] - 0s 178us/step - loss: 0.4410 - acc: 0.8188 - val_loss: 0.3438 - val_acc: 0.8603
Epoch 188/200
712/712 [==================

In [161]:
res = model.predict(test_df.values)

In [162]:
output = [0 for i in range(len(res))]
for i in range(len(res)):
    if res[i][0] >= 0.5:
        output[i] = 0
    else:
        output[i] = 1

In [163]:
submit_file = open("titanic_submit.csv", "w")
file_object = csv.writer(submit_file)
file_object.writerow(["PassengerId", "Survived"])
file_object.writerows(zip(ids, output))
submit_file.close()